In [102]:
import pandas as pd

df = pd.read_csv("Gemini_BTCUSD_1hr.csv", skiprows=1)

In [103]:
df.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1647226800000,2022-03-14 03:00:00,BTCUSD,38055.23,38073.96,38023.12,38060.13,0.868827
1,1647223200000,2022-03-14 02:00:00,BTCUSD,38076.24,38186.59,38009.17,38055.23,10.429404
2,1647219600000,2022-03-14 01:00:00,BTCUSD,37850.60,38348.52,37850.60,38076.24,62.900385
3,1647216000000,2022-03-14 00:00:00,BTCUSD,37791.31,37886.84,37578.09,37850.60,45.794943
4,1647212400000,2022-03-13 23:00:00,BTCUSD,37748.40,37905.78,37599.00,37791.31,46.112435


In [104]:
import numpy as np
#(max'-min')/(max-min)*(value-max)+max'


data = df.to_numpy(na_value=35000)

print(data.size)
print(data[0][0])
max_iter = len(df) 
for k in range(3, 7):
    if k == 6:
        continue
    max_val = -1
    min_val = 9999999
    for i in range(max_iter):
        val = data[i - 1][k]
        if val > max_val:
            max_val = val
        if val < max_val:
            min_val = val
    for i in range(max_iter):
        data[i][k] = float(1/(max_val - min_val) * (data[i][k] - max_val) + 1)
    print(data[2][k])
    
print(data[2][5])

In [105]:
data_c = data[:, 6]
print(data_c)

[38060.13 38055.23 38076.24 ... 244.92 245.0 245.0]


In [106]:

data = np.delete(data, 0, axis=1)
data = np.delete(data, 0, axis=1)
data = np.delete(data, 0, axis=1)


In [107]:
#data = data[0:10000]

In [108]:
data = np.delete(data, 3, axis=1)

In [109]:
import torch
#print(type(data[2][4]))

print(data[10])
print(data[1][1])
print(data_c)
d_t = torch.from_numpy(data.astype(np.float32))
d_c = torch.from_numpy(data_c.astype(np.float32))
d_c.shape
print(d_c)

[38933.87 39191.7 38893.17 16.11749363]
38186.59
[38060.13 38055.23 38076.24 ... 244.92 245.0 245.0]
tensor([38060.1289, 38055.2305, 38076.2383,  ...,   244.9200,   245.0000,
          245.0000])


In [110]:
n_samples = d_t.shape[0]
n_val = int(.2 * n_samples)
shuffled_indices = torch.randperm(n_samples).numpy()
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [111]:
train_d = d_t[train_indices]
train_d_c = d_c[train_indices]
seq_length = 4
val_d = d_t[val_indices]
val_d_c = d_c[val_indices]

In [112]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
       # self.features = features
        #self.target = target
        self.sequence_length = sequence_length
        self.y = target
        self.X = features

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

    

In [113]:
i = 27
sequence_length = 60
print(val_d_c)
train_dataset = SequenceDataset(
    train_d,
    target=train_d_c,
    features=train_d,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    val_d,
    target=val_d_c,
    features=val_d,
    sequence_length=sequence_length
)
X, y = train_dataset[i]
print(y)

tensor([ 8126.1499, 39487.3086,   428.6900,  ..., 49094.3008,  4030.0000,
        58877.8398])
tensor(8420.)


In [114]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [115]:
from torch import nn
device = torch.device('cuda:0')
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0.to(device), c0.to(device)))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [116]:
learning_rate = 1e-4
num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=4, hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print('done')

done


In [119]:
import torch.nn as nn
import torch.optim as optim
print(torch.cuda.is_available())
import time
device = torch.device('cuda:0')
kwargs = {'num_workers': 1, 'pin_memory': True}
#model = nn.Sequential(nn.Linear(4, 512), nn.Tanh(), nn.Linear(512, 256), nn.Tanh(), nn.Linear(256, 1))
model.to(device)
loss_fn = nn.MSELoss()
n_epochs = 50

start = time.time()
model.train()
for epoch in range(n_epochs):
    for X, y in train_loader:
        train_d = X.to(device)
        train_d_c = y.to(device)

        outputs = model(train_d)
        loss = loss_fn(outputs, train_d_c)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
end = time.time()
print(end - start)

True
Epoch: 0, Loss: 445141280.000000
Epoch: 1, Loss: 961471040.000000
Epoch: 2, Loss: 242153936.000000
Epoch: 3, Loss: 591309248.000000
Epoch: 4, Loss: 588216064.000000
Epoch: 5, Loss: 376858368.000000
Epoch: 6, Loss: 431163104.000000
Epoch: 7, Loss: 611583936.000000
Epoch: 8, Loss: 552578688.000000
Epoch: 9, Loss: 366579008.000000
Epoch: 10, Loss: 524777920.000000
Epoch: 11, Loss: 592676352.000000
Epoch: 12, Loss: 671352128.000000
Epoch: 13, Loss: 456253344.000000
Epoch: 14, Loss: 363682464.000000
Epoch: 15, Loss: 354918688.000000
Epoch: 16, Loss: 527134720.000000
Epoch: 17, Loss: 407356992.000000
Epoch: 18, Loss: 461302848.000000
Epoch: 19, Loss: 477014400.000000
Epoch: 20, Loss: 595451328.000000
Epoch: 21, Loss: 445706144.000000
Epoch: 22, Loss: 262693152.000000
Epoch: 23, Loss: 349245056.000000
Epoch: 24, Loss: 455076864.000000
Epoch: 25, Loss: 361098144.000000
Epoch: 26, Loss: 788255872.000000
Epoch: 27, Loss: 729857664.000000
Epoch: 28, Loss: 319252640.000000
Epoch: 29, Loss: 62

In [120]:
model.eval()

with torch.no_grad():
    for X, y in test_loader:
        val_d = X.to(device) 
        val_d_c = y.to(device)

        outputs = model(val_d)

    k = 0
    ttl = 0
    for i in range(len(outputs)):
        #print(outputs[i]/val_d_c[i])
        #print("\n")
        ttl += outputs[i]/val_d_c[i]
        k+=1
    print(ttl/k)

tensor(0.0128, device='cuda:0')
